<a href="https://colab.research.google.com/github/srilamaiti/ml_works/blob/main/conact_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.1 MB/s eta 0:00:00


In [6]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=0e1077d9d25399423769d0f7422d726ff61a4ba02db40a68793a399c65ad303b
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


In [12]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
from transformers import *
from tensorflow.keras import layers
import pandas as pd
import re
import emoji

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import TreebankWordTokenizer
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from transformers import *

In [4]:
def CNN_BERT_Glove(model_hf2, embedding_matrix,seed_value, num_classes):
        
    input_ids_in = tf.keras.layers.Input(shape=(250,), name='input_token', dtype='int32')
    input_masks_in = tf.keras.layers.Input(shape=(250,), name='masked_token', dtype='int32') 
    embedding_layer = model_hf2([input_ids_in,input_masks_in])[1][12]
    CNN_input = tf.keras.layers.Input(shape=(250,), name='CNN_input') 
    embedding_layer2 = layers.Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1], weights=[embedding_matrix], input_length=250 , trainable=False)(CNN_input)
    

    query_seq_encoding = tf.keras.layers.Conv1D(filters=100,kernel_size=3,padding='same')(embedding_layer2)
    query_seq_encoding = tf.keras.layers.GlobalMaxPooling1D()(query_seq_encoding)
    query_seq_encoding2 = tf.keras.layers.GlobalAveragePooling1D()(embedding_layer)
    input_layer = tf.keras.layers.Concatenate()([query_seq_encoding, query_seq_encoding2])
    x = layers.Dropout(0.5, seed=seed_value)(input_layer)
    o = layers.Dense(num_classes, activation='softmax')(x)


    
    model_cnn2 = tf.keras.Model(inputs=[input_ids_in, input_masks_in, CNN_input], outputs=o)
    loss = tf.keras.losses.SparseCategoricalCrossentropy()    
    optimizer='SGD'

    metric = tf.keras.metrics.SparseCategoricalAccuracy('sparse_categorical_accuracy')
    model_hf2.trainable = False
    model_cnn2.compile(optimizer=optimizer, loss=loss, metrics=[metric])
    return model_cnn2


def CNN_BERT_Glovetwitt(model_hf2, embedding_matrix,seed_value,num_classes, train_inputs_v01, valid_inputs_v01, test_inputs_v01, train_masks_v01, valid_masks_v01, test_masks_v01,train_data, valid_data, test_data ,y_train, y_valid, y_test):
    
    my_callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)]
    
    tf.compat.v1.reset_default_graph()
    cnnmodel = CNN_BERT_Glove(model_hf2,embedding_matrix, seed_value, num_classes)
    cnnmodel.fit([train_inputs_v01, train_masks_v01, train_data], y_train, validation_data =([valid_inputs_v01, valid_masks_v01,valid_data],y_valid), epochs=15,verbose=0, batch_size=16,callbacks=my_callbacks)
    predictions =  cnnmodel.predict([test_inputs_v01, test_masks_v01,test_data])
    yhat = np.argmax(predictions, axis=1)
    acc2 = accuracy_score(y_test, yhat)
    f1=f1_score(y_test, yhat,average='macro' )
    print('CNN(BERT+Glove)  Accuracy: %.5f' % acc2)
    print('CNN(BERT+Glove)  F1: %.5f' % f1)
        




def lstm_model1(model_hf2, embedding_matrix,seed_value,num_classes):
        
    input_ids_in = tf.keras.layers.Input(shape=(250,), name='input_token', dtype='int32')
    input_masks_in = tf.keras.layers.Input(shape=(250,), name='masked_token', dtype='int32') 
    embedding_layer = model_hf2([input_ids_in,input_masks_in])[1][12]
    CNN_input = tf.keras.layers.Input(shape=(250,), name='CNN_input') 
    embedding_layer2 = layers.Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1], weights=[embedding_matrix], input_length=250 , trainable=False)(CNN_input)
    

    query_seq_encoding = tf.keras.layers.LSTM(units=100)(embedding_layer2)
    query_seq_encoding2 = tf.keras.layers.GlobalAveragePooling1D()(embedding_layer)
    input_layer = tf.keras.layers.Concatenate()([query_seq_encoding, query_seq_encoding2])
    x = layers.Dropout(0.5, seed=seed_value)(input_layer)
    o = layers.Dense(num_classes, activation='softmax')(x)

    
    model_lstm = tf.keras.Model(inputs=[input_ids_in, input_masks_in, CNN_input], outputs=o)
    loss = tf.keras.losses.SparseCategoricalCrossentropy()    
    optimizer='SGD'

    metric = tf.keras.metrics.SparseCategoricalAccuracy('sparse_categorical_accuracy')
    model_hf2.trainable = False
    model_lstm.compile(optimizer=optimizer, loss=loss, metrics=[metric])
    return model_lstm




def LSTM_BERT_Glovetwitt(model_hf2, embedding_matrix, seed_value,num_classes, train_inputs_v01, valid_inputs_v01, test_inputs_v01, train_masks_v01, valid_masks_v01, test_masks_v01,train_data, valid_data, test_data ,y_train, y_valid, y_test):

    my_callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)]
    tf.compat.v1.reset_default_graph()
    lstm_model = lstm_model1(model_hf2,embedding_matrix, seed_value, num_classes)
    lstm_model.fit([train_inputs_v01, train_masks_v01, train_data], y_train, validation_data =([valid_inputs_v01, valid_masks_v01,valid_data],y_valid), epochs=15,verbose=0, batch_size=16,callbacks=my_callbacks)
    predictions =  lstm_model.predict([test_inputs_v01, test_masks_v01,test_data])
    yhat = np.argmax(predictions, axis=1)
    acc2 = accuracy_score(y_test, yhat)
    f1=f1_score(y_test, yhat,average='macro' )
    print('LSTM(BERT+Glove)  Accuracy: %.5f' % acc2)
    print('LSTM(BERT+Glove) F1: %.5f' % f1)


In [13]:
def load_dict_smileys():

    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

# self defined contractions
def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }




def remove_redundant_punct(text,redundant_punct_pattern):
    text_ = text
    result = re.search(redundant_punct_pattern, text)
    dif = 0
    while result:
        sub = result.group()
        sub = sorted(set(sub), key=sub.index)
        sub = ' ' + ''.join(list(sub)) + ' '
        text = ''.join((text[:result.span()[0]+dif], sub, text[result.span()[1]+dif:]))
        text_ = ''.join((text_[:result.span()[0]], text_[result.span()[1]:])).strip()
        dif = abs(len(text) - len(text_))
        result = re.search(redundant_punct_pattern, text_)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


def preprocess(text):
    regex_url_step1 = r'(?=http)[^\s]+'
    regex_url_step2 = r'(?=www)[^\s]+'
    regex_url = r'(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)'
    regex_mention = r'@[\w\d]+'
    regex_email = r'\S+@\S+'
    redundant_punct_pattern = r'([!\"#\$%\'\(\)\*\+,\.:;\-<=·>?@\[\\\]\^_ـ`{\|}~—٪’،؟`୍“؛”ۚ【»؛\s+«–…‘]{2,})'

    text=str(text)
    processing_tweet = re.sub('ـ', '', text)
    processing_tweet= processing_tweet.lower()
    processing_tweet = re.sub('[«»]', ' " ', processing_tweet)
    processing_tweet = re.sub(regex_url_step1, '[link]', processing_tweet)
    processing_tweet = re.sub(regex_url_step2, '[link]', processing_tweet)
    processing_tweet = re.sub(regex_url, '[link]', processing_tweet)
    processing_tweet = re.sub(regex_email, '[email]', processing_tweet)
    processing_tweet = re.sub(regex_mention, '[user]', processing_tweet)
    processing_tweet = re.sub('…', r'\.', processing_tweet).strip()
    processing_tweet = remove_redundant_punct(processing_tweet, redundant_punct_pattern)
    processing_tweet = re.sub(r'\[ link \]|\[ link\]|\[link \]', ' [link] ', processing_tweet)
    processing_tweet = re.sub(r'\[ email \]|\[ email\]|\[email \]', ' [email] ', processing_tweet)
    processing_tweet = re.sub(r'\[ user \]|\[ user\]|\[user \]', ' [user] ', processing_tweet)
    processing_tweet = re.sub("(.)\\1{2,}", "\\1", processing_tweet)
    processing_tweet=strip_emoji(processing_tweet)

    search = ['_','\\','\n','-', ',','/' ,'.','\t','?','!','+','*','\'','|','#', '$','%']
    replace = [' ', ' ',' ',' ', ' ',' ', ' ',' ',' ',' ',' ',' ',' ',' ', ' ', ' ',' ']
    #remove numbers
    processing_tweet = re.sub(r'\d+', '', processing_tweet)
    processing_tweet = ' '.join(re.sub("[\n\.\,\"\!\?\:\;\-\=\؟]", " ", processing_tweet).split())
    processing_tweet = ' '.join(re.sub("[\_]", " ", processing_tweet).split())
    processing_tweet = re.sub(r'[^\x00-\x7F]+',' ', processing_tweet)

    for i in range(0, len(search)):
        processing_tweet = processing_tweet.replace(search[i], replace[i])

    return processing_tweet.strip()

def strip_emoji(text):
    new_text = re.sub(emoji.get_emoji_regexp(), r" ", text)
    return new_text

def get_preprocessed_data(training_path_text,training_path_labels,validation_path_text,validation_path_labels,testing_path_text,testing_path_labels):
    
    #Train
    data_file_train_text =open(training_path_text, "r", encoding='utf-8')
    datatable_arabic_train_text = [preprocess(str(line)) for line in data_file_train_text.read().splitlines()]
    data_file_train_labels =open(training_path_labels, "r", encoding='utf-8')
    datatable_arabic_train_labels = [int(line) for line in data_file_train_labels.read().splitlines()]
    d = {'tweet': datatable_arabic_train_text, 'label': datatable_arabic_train_labels}
    df_train = pd.DataFrame(data=d)
    
    
    #Val
    data_file_valid_text =open(validation_path_text, "r", encoding='utf-8')
    datatable_arabic_valid_text = [preprocess(str(line)) for line in data_file_valid_text.read().splitlines()]
    data_file_valid_labels =open(validation_path_labels, "r", encoding='utf-8')
    datatable_arabic_valid_labels = [int(line) for line in data_file_valid_labels.read().splitlines()]
    d2 = {'tweet': datatable_arabic_valid_text, 'label': datatable_arabic_valid_labels}
    df_valid = pd.DataFrame(data=d2)
    
    
    #Test
    data_file_test_text =open(testing_path_text, "r", encoding='utf-8')
    datatable_arabic_test_text = [preprocess(str(line))  for line in data_file_test_text.read().splitlines()]
    data_file_test_labels =open(testing_path_labels, "r", encoding='utf-8')
    datatable_arabic_test_labels = [int(line) for line in data_file_test_labels.read().splitlines()]
    d3 = {'tweet': datatable_arabic_test_text, 'label': datatable_arabic_test_labels}
    df_test = pd.DataFrame(data=d3)
    
    return df_train, df_valid, df_test


In [14]:
def toknize_text(data_train, data_valid, data_test):
    new_tokenizer = Tokenizer()
    new_tokenizer.fit_on_texts(data_train+data_valid+data_test)
    data_train= new_tokenizer.texts_to_sequences(data_train)
    data_train= pad_sequences(data_train, padding='post', maxlen=250)
    data_valid= new_tokenizer.texts_to_sequences(data_valid)
    data_valid= pad_sequences(data_valid, padding='post', maxlen=250)
    data_test= new_tokenizer.texts_to_sequences(data_test)
    data_test= pad_sequences(data_test, padding='post', maxlen=250)
    vocab_size = len(new_tokenizer.word_index) + 1
    return data_train,data_valid,data_test, vocab_size,new_tokenizer


def get_final_data(xTrain,xValid,xTest):
    data_train=[]
    data_valid=[]
    data_test=[]
    for sentence in list(xTrain):
        sentence= TreebankWordTokenizer().tokenize(sentence)
        data_train.append(sentence)
    for sentence in list(xValid):
        sentence= TreebankWordTokenizer().tokenize(sentence)
        data_valid.append(sentence)
    for sentence in list(xTest):
        sentence= TreebankWordTokenizer().tokenize(sentence)
        data_test.append(sentence)

    train_data, valid_data, test_data, n_vocab, n_tokenizer= toknize_text(data_train, data_valid,data_test)
    train_data=np.array(train_data)
    valid_data=np.array(valid_data)
    test_data=np.array(test_data)
    return train_data, valid_data, test_data, n_vocab, n_tokenizer



def getEmbeddingMatrix(n_vocab, n_tokenizer,pretrainedEmbedding):
    i=0
    embedding_matrix= zeros((n_vocab, 100))
    oov_words=[]
    for word , index in n_tokenizer.word_index.items():
        embedding_vector=None
        try:
            if word in pretrainedEmbedding.wv:
                embedding_matrix[index] = pretrainedEmbedding[word]
            else: 
                i+=1
                oov_words.append(word)
        except:
            continue
    return embedding_matrix, oov_words,i


def get_glove_toknized_text(df_train,df_valid, df_test, golve_twitter):
    xTrain=df_train['tweet']
    y_train=df_train.label.values
    
    xValid=df_valid['tweet']
    y_valid=df_valid.label.values
    
    xTest=df_test['tweet']
    y_test=df_test.label.values
    
    train_data, valid_data, test_data, n_vocab, n_tokenizer= get_final_data(xTrain,xValid,xTest)
    embedding_matrix, oov_words,count_oov=getEmbeddingMatrix(n_vocab, n_tokenizer, golve_twitter)
    
    return train_data, valid_data, test_data, n_vocab, n_tokenizer, embedding_matrix, oov_words,count_oov,y_train, y_valid, y_test

def prepareBertInput(tokenizer,sentences):
        attention_mask=[]
        input_ids=[]
        tokenized = sentences.apply((lambda x: tokenizer.encode(str(x), add_special_tokens=True)))
        max_len = 0
        for i in tokenized.values:
            if len(i) > max_len:
                max_len = len(i)
        for sentence in sentences:
            tokenized2=tokenizer.encode_plus(str(sentence),  max_length=250, pad_to_max_length=True,add_special_tokens=True)
            attention_mask.append(tokenized2['attention_mask'])
            input_ids.append(tokenized2['input_ids'])
    
        return input_ids , attention_mask, max_len,tokenized 

def get_bert_inputs(df_train,df_valid, df_test):
    ##BERT 
    bert_tokenizer_transformer_v01 = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    
    xTrain=df_train['tweet']
    xValid=df_valid['tweet']
    xTest=df_test['tweet']
    
    train_inputs_v01, train_masks_v01, max_len_v01,tokenized =prepareBertInput(bert_tokenizer_transformer_v01,xTrain )
    valid_inputs_v01, valid_masks_v01, max_len_v01_valid,tokenized_valid =prepareBertInput(bert_tokenizer_transformer_v01,xValid )
    test_inputs_v01, test_masks_v01, max_len_v01_test,tokenized_test =prepareBertInput(bert_tokenizer_transformer_v01,xTest )
    
    
    train_inputs_v01=tf.constant(train_inputs_v01)
    valid_inputs_v01=tf.constant(valid_inputs_v01)
    test_inputs_v01=tf.constant(test_inputs_v01)
    train_masks_v01=tf.constant(train_masks_v01)
    valid_masks_v01=tf.constant(valid_masks_v01)
    test_masks_v01=tf.constant(test_masks_v01)
    
    return train_inputs_v01, valid_inputs_v01, test_inputs_v01, train_masks_v01, valid_masks_v01, test_masks_v01